In [1]:
import pandas as pd
import numpy as np

## Features engineering de pageviews

### Carga de datos

In [2]:
pageviews = pd.read_pickle("../data/pageviews_total_formateado.pkl")

---

### Cargo features generados hasta el momento


In [3]:
# Trimestre 2 - 3 - 4 para los que tengo datos anteriores
CANT_TRIMESTRES = 4

In [4]:
ventanas = {}
vent_no_encontradas =[]

In [5]:
print("Cargando features")
for ventana_nro in range(1,CANT_TRIMESTRES) : #Trimestres del 1 al 3
    try:
        ventanas[ventana_nro] = pd.read_csv("../data/features/ventana_{}_pageviews.csv".format(ventana_nro), index_col=0)
    except FileNotFoundError:
        print("No se encontraron los features, se generarán los dataframe requeridos")
        vent_no_encontradas.append(ventana_nro)
    continue
    
for ventana_nro in vent_no_encontradas:    
    ##Recolecto los features del trimestre anterior al que quiero predecir
    condicion = pageviews["trimestre"] == (ventana_nro)
    
    ventanas[ventana_nro] = pageviews.loc[condicion]["USER_ID"] \
        .drop_duplicates().to_frame().set_index("USER_ID").copy()

Cargando features
No se encontraron los features, se generarán los dataframe requeridos
No se encontraron los features, se generarán los dataframe requeridos
No se encontraron los features, se generarán los dataframe requeridos


### Set para predecir

In [6]:
ventana_predecir = {}

In [7]:
try:
    print("Cargando features para predecir")
    ventana_predecir = pd.read_csv("../data/features/ventana_4_pageviews.csv")
except FileNotFoundError:
    print("No se encontro el feature deseado, se generara el dataframe requerido")
    ventana_predecir[4] = pd.read_csv("../data/users.csv", dtype = {"USER_ID":"category"})
    ventana_predecir[4].set_index("USER_ID",inplace = True)

Cargando features para predecir
No se encontro el feature deseado, se generara el dataframe requerido


---

### Diccionario de features

In [8]:
features = {}

In [9]:
features_ya_generados = {}

for ventana_nro in range (1, CANT_TRIMESTRES):
    
    features_ya_generados[ventana_nro] = list(ventanas[ventana_nro].columns.values)
    
features_ya_generados[4] = list(ventana_predecir[4].columns.values)



---

### Creación de features

In [10]:
def cantidad_de_paginas_vistas(dataframe):
    return dataframe.groupby("USER_ID").agg({"FEC_EVENT":"count"})\
                    .rename(columns = {"FEC_EVENT":"cantidad_paginas_vistas_trimestre"})

In [11]:
features["cantidad_paginas_vistas_trimestre"] = (cantidad_de_paginas_vistas,["cantidad_paginas_vistas_trimestre"])

---

## Generacion de features

In [18]:
def generar_feature_en_ventanas(dataframe, generador_feature, destino, nro_ventana):
    """ 
    Esta funcion calcula un feature para un trimestre o ventana en especifico, lo une con los ya calculados
    y lo guarda en el diccionario 'destino'.    
    dataframe: dataframe de datos de donde se calcularan los features, en este caso pageviews
    generador_feature: funcion que calculara el feature y devolvera su resultado
    destino: diccionario que contiene las ventanas de features calculados
    """
    dataframe = dataframe.loc[dataframe["trimestre"] == nro_ventana]
    feature = generador_feature(dataframe)
    destino[nro_ventana] = destino[nro_ventana].merge(feature, left_index = True, right_index= True, how = 'left')

### Genero los features


In [13]:
#Posicion dentro del diccionario de features que ocupa la lista de columnas generadas por el mismo
POS_COLUMNAS_DEL_FEATURE = 1  
#Posicion dentro del diccionario de features que ocupa la funcion que lo genera.
POS_FUNCION_DEL_FEATURE = 0

def generar_features_faltantes(dataframe,destino,nro_ventana):
    """
    
    """
    
    for nombre_feature in features:    
        #Obtengo los nombres de las columnas generadas por la funcion que calcula el feature
        #Una funcion puede devolver mas de un feature por lo que obtengo una lista de valores
        columnas_generadas = features[nombre_feature][POS_COLUMNAS_DEL_FEATURE] 
        feature_ya_generado = False
        
        for columna in columnas_generadas: #Verifico si los features ya fueron generados o no
            
            if columna in features_ya_generados[nro_ventana]: 
                feature_ya_generado = True
                break
    
        if feature_ya_generado:
            continue

        #Si no fueron generados, los genero:    
        generar_feature_en_ventanas(dataframe, features[nombre_feature][POS_FUNCION_DEL_FEATURE], destino,nro_ventana)
                                
        features_ya_generados[nro_ventana].extend(features[nombre_feature][POS_COLUMNAS_DEL_FEATURE])

In [19]:
for ventana_nro in range(1, CANT_TRIMESTRES ):
    generar_features_faltantes(pageviews, ventanas, ventana_nro)

generar_features_faltantes(pageviews, ventana_predecir,4)    

### Guardado

In [25]:
for ventana_nro in range(1, CANT_TRIMESTRES ):
    ventanas[ventana_nro].to_csv("../data/features/ventana_{}_pageviews.csv".format(ventana_nro))

ventana_predecir[4].to_csv("../data/features/predecir_pageviews.csv")